# Datenexploration für Schlafqualität-Vorhersage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn')

## 1. Laden und Überblick des Datensatzes

In [ ]:
# Daten laden
df = pd.read_csv('../data/sleep_health_metrics.csv')
print(df.head())

# Informationen über den Datensatz
print("\nDatensatzinformationen:")
print(df.info())

# Überprüfung auf fehlende Werte
print("\nFehlende Werte:")
print(df.isnull().sum())

# Gedanken: Es ist wichtig zu überprüfen, ob es fehlende Werte gibt und wie wir damit umgehen sollten.

## 2. Statistische Zusammenfassung

In [ ]:
# Grundlegende statistische Informationen
print(df.describe())

# Gedanken: Die statistische Zusammenfassung gibt uns einen ersten Einblick in die Verteilung der Daten.
# Wir sollten auf Ausreißer und ungewöhnliche Werte achten.

## 3. Visualisierung der Verteilungen

In [ ]:
# Histogramme für numerische Variablen
numeric_columns = df.select_dtypes(include=[np.number]).columns
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.histplot(data=df, x=col, kde=True, ax=axes[i])
    axes[i].set_title(f'Verteilung von {col}')

plt.tight_layout()
plt.show()

# Gedanken: Die Verteilungen helfen uns, Ausreißer zu identifizieren und zu verstehen,
# ob Transformationen der Daten notwendig sein könnten.

## 4. Korrelationsanalyse

In [ ]:
# Korrelationsmatrix
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_columns].corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Korrelationsmatrix der numerischen Variablen')
plt.show()

# Gedanken: Die Korrelationsmatrix hilft uns, Beziehungen zwischen den Variablen zu erkennen.
# Starke Korrelationen könnten auf Multikollinearität hinweisen, was bei der Modellierung berücksichtigt werden sollte.

## 5. Identifizierung möglicher Ausreißer

In [ ]:
# Boxplots für numerische Variablen
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.boxplot(data=df, y=col, ax=axes[i])
    axes[i].set_title(f'Boxplot von {col}')

plt.tight_layout()
plt.show()

# Gedanken: Boxplots sind nützlich, um Ausreißer visuell zu identifizieren.
# Wir sollten überlegen, wie wir mit diesen Ausreißern umgehen wollen (z.B. entfernen, transformieren oder beibehalten).

## 6. Zusammenfassung und nächste Schritte

Basierend auf unserer Datenexploration können wir folgende Schlüsse ziehen und nächste Schritte planen:

1. Datenqualität: Überprüfen Sie, ob fehlende Werte behandelt werden müssen.
2. Ausreißer: Identifizieren Sie signifikante Ausreißer und entscheiden Sie, wie damit umzugehen ist.
3. Verteilungen: Betrachten Sie, ob Datentransformationen (z.B. Log-Transformation) für schiefe Verteilungen sinnvoll sind.
4. Korrelationen: Berücksichtigen Sie starke Korrelationen bei der Modellauswahl und -entwicklung.
5. Feature Engineering: Überlegen Sie, ob neue Features aus den vorhandenen Daten erstellt werden können.
6. Kategorische Variablen: Führen Sie eine detailliertere Analyse der kategorischen Variablen durch (z.B. Häufigkeitsverteilungen).

Der nächste Schritt könnte die Datenvorbereitung sein, gefolgt von der Auswahl und dem Training eines geeigneten Modells zur Vorhersage der Schlafqualität.